In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
sub = pd.read_csv("data/sample_submission.csv")

In [3]:
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [4]:
train.columns

Index(['id', 'cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13', 'target'],
      dtype='object')

In [5]:
y = train.target
train.drop(['target', 'id'], axis = 1, inplace = True)
test.drop(['id'], axis = 1, inplace = True)
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [6]:
test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,E,I,...,0.476739,0.376350,0.337884,0.321832,0.445212,0.290258,0.244476,0.087914,0.301831,0.845702
1,A,B,A,C,B,C,A,E,C,H,...,0.285509,0.860046,0.798712,0.835961,0.391657,0.288276,0.549568,0.905097,0.850684,0.693940
2,B,A,A,A,B,B,A,E,D,K,...,0.697272,0.683600,0.404089,0.879379,0.275549,0.427871,0.491667,0.384315,0.376689,0.508099
3,B,B,A,C,B,D,A,E,A,N,...,0.719306,0.777890,0.730954,0.644315,1.024017,0.391090,0.988340,0.411828,0.393585,0.461372
4,B,B,A,C,B,C,A,E,C,F,...,0.313032,0.431007,0.390992,0.408874,0.447887,0.390253,0.648932,0.385935,0.370401,0.900412


In [7]:
print("Train shape: ", train.shape, "\nTest shape: ", test.shape)

Train shape:  (300000, 24) 
Test shape:  (200000, 24)


In [8]:
from sklearn.preprocessing import OrdinalEncoder
cat_cols = [col for col in train.columns if 'cat' in col]

X = train.copy()
X_test = test.copy()
enc = OrdinalEncoder()
X[cat_cols] = enc.fit_transform(train[cat_cols])
X_test[cat_cols] = enc.transform(test[cat_cols])
X.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,4.0,2.0,13.0,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,1.0,1.0,0.0,0.0,1.0,3.0,0.0,5.0,0.0,14.0,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,0.0,0.0,0.0,2.0,1.0,3.0,0.0,3.0,0.0,5.0,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,1.0,1.0,0.0,2.0,1.0,3.0,0.0,4.0,2.0,10.0,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,0.0,0.0,0.0,2.0,1.0,3.0,0.0,4.0,0.0,13.0,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


In [38]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# Model hyperparameters
xgb_params = {
    'n_estimators': 10000,
    'learning_rate': 0.1,
    'max_depth': 2,
    'tree_method':'gpu_hist'
}
"""xgb_params = {'n_estimators': 10000,
              'learning_rate': 0.35,
              'subsample': 0.926,
              'colsample_bytree': 0.84,
              'max_depth': 2,
              'booster': 'gbtree', 
              'reg_lambda': 35.1,
              'reg_alpha': 34.9,
              'random_state': 42,
              'n_jobs': 4}"""

"xgb_params = {'n_estimators': 10000,\n              'learning_rate': 0.35,\n              'subsample': 0.926,\n              'colsample_bytree': 0.84,\n              'max_depth': 2,\n              'booster': 'gbtree', \n              'reg_lambda': 35.1,\n              'reg_alpha': 34.9,\n              'random_state': 42,\n              'n_jobs': 4}"

In [39]:
#Setting the kfold parameters
kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

oof_preds = np.zeros((X.shape[0],))
preds = 0
model_fi = 0
mean_rmse = 0

for num, (train_id, valid_id) in enumerate(kf.split(X)):
    X_train, X_valid = X.loc[train_id], X.loc[valid_id]
    y_train, y_valid = y.loc[train_id], y.loc[valid_id]
    
    model = XGBRegressor(**xgb_params)
    model.fit(X_train, y_train,
             verbose = False,
             eval_set = [(X_train, y_train), (X_valid, y_valid)],
             eval_metric = "rmse",
             early_stopping_rounds = 100)
    
    #Mean of the predictions
    preds += model.predict(X_test) / 10 # Splits
    
    #Mean of feature importance
    model_fi += model.feature_importances_ / 10 #splits
    
    #Out of Fold predictions
    oof_preds[valid_id] = model.predict(X_valid)
    fold_rmse = np.sqrt(mean_squared_error(y_valid, oof_preds[valid_id]))
    print(f"Fold {num} | RMSE: {fold_rmse}")
    
    mean_rmse += fold_rmse / 10
    
print(f"\nOverall RMSE: {mean_rmse}")

Fold 0 | RMSE: 0.7175532124659177
Fold 1 | RMSE: 0.7175386402275418
Fold 2 | RMSE: 0.7166179340692136
Fold 3 | RMSE: 0.7189273238223703
Fold 4 | RMSE: 0.7229659368710675
Fold 5 | RMSE: 0.716058891462996
Fold 6 | RMSE: 0.7192415838829889
Fold 7 | RMSE: 0.7197123507541386
Fold 8 | RMSE: 0.7214418415862143
Fold 9 | RMSE: 0.714420069152877

Overall RMSE: 0.7184477784295327


In [20]:
sub.target = preds
sub.to_csv("submission_3.csv", index = False)